In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import random
import string
import hashlib
import json

#Loading original data
columns_name = ['id', 'date', 'latitude', 'longitude' ]

df = pd.read_csv('../Data_Anom/real_data_set.csv', names = columns_name, sep = '\t')
df['date'] = df['date'].str.slice(stop = 16)
df['date'] = pd.to_datetime(df['date'], format = "%Y-%m-%d %H:%M")
df['week'] = df['date'].dt.isocalendar().week

In [2]:
anon = pd.read_csv('../Data_Anom/THE_437', names = columns_name, sep = '\t')
anon = anon.loc[anon.id != 'DEL']
anon['date'] = anon['date'].str.slice(stop = 16)
anon['date'] = pd.to_datetime(anon['date'], format = "%Y-%m-%d %H:%M")
anon['week'] = anon['date'].dt.isocalendar().week
anon.latitude = anon.latitude.astype(float)
anon.longitude = anon.longitude.astype(float)

In [3]:
print(anon)

           id                date  latitude  longitude  week
0         171 2015-04-13 15:19:00  5.108934  45.636557    16
1         644 2015-03-08 16:15:00  5.146483  45.631887    10
2         761 2015-03-14 18:02:00  4.888703  45.750283    11
3           0 2015-03-05 19:18:00  4.873628  45.784857    10
4         461 2015-03-07 22:38:00  4.842303  45.770993    10
...       ...                 ...       ...        ...   ...
34551844  492 2015-03-10 05:08:00  4.852862  45.738185    11
34551845  285 2015-03-27 18:20:00  4.804872  45.766993    13
34551846  639 2015-04-17 12:06:00  4.589882  45.819353    16
34551847  577 2015-05-11 14:43:00  4.872865  45.782248    20
34551848  503 2015-04-04 16:08:00  4.872170  45.782233    14

[32624236 rows x 5 columns]


In [4]:
print(df)

           id                date  latitude  longitude  week
0           1 2015-03-04 00:35:00  4.870147  45.772140    10
1           1 2015-03-04 00:35:00  4.870218  45.772095    10
2           1 2015-03-04 00:35:00  4.870210  45.772072    10
3           1 2015-03-04 00:35:00  4.870210  45.772072    10
4           1 2015-03-04 00:35:00  4.870210  45.772072    10
...       ...                 ...       ...        ...   ...
34551844  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551845  110 2015-03-12 16:23:00  2.343094  48.891650    11
34551846  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551847  110 2015-03-12 16:23:00  2.343094  48.891649    11
34551848  110 2015-03-12 19:29:00  2.343127  48.891772    11

[34551849 rows x 5 columns]


In [5]:
#List of id origin, id anonym, group of id-week

id_week_ori = df.groupby('id')['week'].unique()
id_week_anon = anon.groupby('id')['week'].unique()

list_id_ori = df["id"].unique()
list_id_anon = anon["id"].unique()

print(np.sort(list_id_ori), len(list_id_ori))
print(np.sort(list_id_anon), len(list_id_anon))

[  1   2   4   5   6   7   8   9  11  13  14  15  16  17  18  21  23  24
  25  26  27  28  29  30  31  32  34  35  36  37  38  39  41  42  43  44
  48  49  50  51  52  53  54  55  58  59  60  62  63  65  66  67  68  69
  70  71  72  73  75  77  78  81  83  84  87  89  98 107 110] 69
['0' '1000' '1002' '1003' '1007' '1009' '1013' '1016' '102' '1021' '1023'
 '1025' '103' '1031' '1033' '1041' '1042' '1045' '1049' '1052' '106' '107'
 '108' '115' '12' '124' '129' '131' '135' '138' '139' '141' '142' '144'
 '145' '15' '151' '156' '158' '16' '162' '163' '167' '170' '171' '174'
 '175' '176' '177' '18' '180' '187' '191' '193' '195' '198' '200' '201'
 '202' '205' '207' '209' '210' '211' '216' '217' '220' '223' '224' '231'
 '233' '235' '236' '237' '239' '240' '241' '246' '251' '253' '255' '256'
 '258' '26' '260' '265' '266' '267' '268' '270' '275' '276' '282' '284'
 '285' '293' '294' '299' '3' '300' '301' '304' '305' '31' '310' '312'
 '316' '32' '320' '321' '322' '325' '328' '34' '340' '341' '342'

In [6]:
#Group id--week--moyen_latitude--moyen_longitude--mean_date--count_line--mean_hour

#Original Table
df['hour'] = df['date'].dt.hour
mean_hour = df.groupby(['id','week'])['hour'].mean()

mean_lat_ori = df.groupby(['id','week'])['latitude'].mean()
mean_long_ori = df.groupby(['id','week'])['longitude'].mean()

mean_date_ori = df.groupby(['id','week'])['date'].mean()
count_ligne_ori = df.groupby(['id','week'])['date'].count()
mean_hour_ori = df.groupby(['id','week'])['hour'].mean()

ori_mean = df[['id','week']].drop_duplicates().reset_index(drop=True)
ori_mean['mean_latitude'] = ori_mean.apply(lambda x: mean_lat_ori[x.id][x.week], axis=1)
ori_mean['mean_longitude'] = ori_mean.apply(lambda x: mean_long_ori[x.id][x.week], axis=1)
ori_mean['mean_date'] = ori_mean.apply(lambda x: mean_date_ori[x.id][x.week], axis=1)
ori_mean['count_line'] = ori_mean.apply(lambda x: count_ligne_ori[x.id][x.week], axis=1)
ori_mean['mean_hour'] = ori_mean.apply(lambda x: mean_hour[x.id][x.week], axis=1)

#Anonymisation Table
anon['hour'] = anon['date'].dt.hour
mean_lat_anon = anon.groupby(['id','week'])['latitude'].mean()
mean_long_anon = anon.groupby(['id','week'])['longitude'].mean()
mean_date_anon = anon.groupby(['id','week'])['date'].mean()
count_ligne_anon = anon.groupby(['id','week'])['date'].count()
mean_hour_anon = anon.groupby(['id','week'])['hour'].mean()

anon_mean = anon[['id','week']].drop_duplicates().reset_index(drop=True)
anon_mean['mean_latitude'] = anon_mean.apply(lambda x: mean_lat_anon[x.id][x.week] , axis=1)
anon_mean['mean_longitude'] = anon_mean.apply(lambda x: mean_long_anon[x.id][x.week], axis=1)
anon_mean['mean_date'] = anon_mean.apply(lambda x: mean_date_anon[x.id][x.week], axis=1)
anon_mean['count_line'] = anon_mean.apply(lambda x: count_ligne_anon[x.id][x.week], axis=1)
anon_mean['mean_hour'] = anon_mean.apply(lambda x: mean_hour_anon[x.id][x.week], axis=1)

In [17]:
ori_mean['new_id'] = ori_mean['id']
scale = anon.shape[0] / df.shape[0]
for i in list_id_ori:
    for w in id_week_ori[i]:
        list_distance = list() 
        mean_lat = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_latitude'].to_list()[0]
        mean_long = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_longitude'].to_list()[0]
        mean_date = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_date'].to_list()[0]
        number_line = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'count_line'].to_list()[0]
        mean_hour = ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'mean_hour'].to_list()[0]
        
        group_week = anon_mean.loc[anon_mean.week ==w,['id','week','mean_latitude','mean_longitude','count_line','mean_date','mean_hour']].reset_index(drop=True)
        for j in range(len(group_week)):
            distance = np.power((group_week.mean_latitude[j] - mean_lat)**2 + (group_week.mean_longitude[j] - mean_long)**2, 1/2)
            
            delta_ratio = np.power(abs((group_week.count_line[j]/anon.shape[0] - number_line/df.shape[0])),4)
            delta_time = np.power(abs((group_week.mean_date[j] - mean_date).total_seconds() / 3600),1)
            
            
            delta_hour = np.power(abs((group_week.mean_hour[j] - mean_hour)) ,1/2)
            
            list_distance.append(distance*delta_ratio*delta_hour*delta_time) #*delta_count*delta_time*distance*delta_hour
            
        index = list_distance.index(min(list_distance))
        l = id_week_anon[group_week.id[index]]# list of week have id anon_mean.id[index] 
        while ((w not in l)):
            list_distance[index] = 'DEL'
            index = list_distance.index(min(list_distance).all())
            l = id_week_anon[group_week.id[index]]
        if (w in l):
            ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'] = group_week.id[index]
        
#print(ori_mean.head())
print(ori_mean.new_id.nunique())

result = {}
count2 = 0
for i in list_id_ori:
    result[str(i)] = {}
    for w in id_week_ori[i]:
        result[str(i)]['2015'+'-'+str(w)] = [str(ori_mean.loc[(ori_mean.id == i)&(ori_mean.week ==w),'new_id'][count2])]
        count2 +=1
print(result)

339
{'1': {'2015-10': ['239'], '2015-11': ['478'], '2015-12': ['202'], '2015-13': ['220'], '2015-14': ['325'], '2015-15': ['550'], '2015-16': ['500'], '2015-17': ['987'], '2015-18': ['680'], '2015-19': ['300'], '2015-20': ['732']}, '2': {'2015-10': ['804'], '2015-11': ['782'], '2015-12': ['612'], '2015-13': ['645'], '2015-14': ['258'], '2015-15': ['546'], '2015-16': ['419'], '2015-17': ['364'], '2015-18': ['932'], '2015-19': ['457'], '2015-20': ['637']}, '107': {'2015-10': ['467'], '2015-11': ['694'], '2015-12': ['677']}, '98': {'2015-10': ['439'], '2015-11': ['473']}, '21': {'2015-10': ['627'], '2015-11': ['947']}, '5': {'2015-10': ['211'], '2015-11': ['265']}, '9': {'2015-10': ['461'], '2015-11': ['461'], '2015-12': ['454'], '2015-13': ['921'], '2015-14': ['722'], '2015-15': ['630'], '2015-16': ['857'], '2015-19': ['255']}, '6': {'2015-10': ['644'], '2015-11': ['73'], '2015-12': ['1042'], '2015-13': ['961'], '2015-14': ['918'], '2015-15': ['260'], '2015-16': ['171']}, '7': {'2015-10'

In [16]:
jsonData = json.dumps(result, indent=4)
print(jsonData)

{
    "1": {
        "2015-10": [
            "239"
        ],
        "2015-11": [
            "478"
        ],
        "2015-12": [
            "202"
        ],
        "2015-13": [
            "220"
        ],
        "2015-14": [
            "325"
        ],
        "2015-15": [
            "550"
        ],
        "2015-16": [
            "500"
        ],
        "2015-17": [
            "987"
        ],
        "2015-18": [
            "680"
        ],
        "2015-19": [
            "300"
        ],
        "2015-20": [
            "732"
        ]
    },
    "2": {
        "2015-10": [
            "804"
        ],
        "2015-11": [
            "782"
        ],
        "2015-12": [
            "612"
        ],
        "2015-13": [
            "645"
        ],
        "2015-14": [
            "258"
        ],
        "2015-15": [
            "546"
        ],
        "2015-16": [
            "419"
        ],
        "2015-17": [
            "364"
        ],
        "2015-18": [